In [6]:
#
# classifer notebook
#
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    
#
# PARAMS
#

batch_size = 4

input_size = 2 * (21 * 3) + 12 #138
hidden_size = 96
num_classes = 3

learning_rate = .01
num_epochs = 100


#
# INITIAL DATA
#

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/landmarks_indexed.csv",
                              transform=transformations)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


Epoch [10/100], Loss: 1.5451, Running Loss (per N epoch): 82.9215
Epoch [20/100], Loss: 1.3247, Running Loss (per N epoch): 75.7985
Epoch [30/100], Loss: 1.3254, Running Loss (per N epoch): 74.0952
Epoch [40/100], Loss: 0.7062, Running Loss (per N epoch): 72.2005
Epoch [50/100], Loss: 1.0162, Running Loss (per N epoch): 70.0758
Epoch [60/100], Loss: 1.0203, Running Loss (per N epoch): 65.7861
Epoch [70/100], Loss: 0.9391, Running Loss (per N epoch): 65.3044
Epoch [80/100], Loss: 1.0829, Running Loss (per N epoch): 65.4164
Epoch [90/100], Loss: 0.4583, Running Loss (per N epoch): 63.6583
Epoch [100/100], Loss: 1.4868, Running Loss (per N epoch): 62.2822


In [12]:
#
# EXPORT
#

dummy_input = torch.zeros(input_size)
model.load_state_dict(torch.load('./model_overfit.pt'))
torch.onnx.export(model, dummy_input, 'onnx_model.onnx', export_params=True,
                  input_names = ['landmarks'], output_names = ['class'], verbose=True)

#import onnx
#onnx_model = onnx.load("./onnx_model.onnx")
#onnx.checker.check_model(onnx_model)


Exported graph: graph(%landmarks : Float(138, strides=[1], requires_grad=0, device=cpu),
      %fc1.bias : Float(96, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(3, strides=[1], requires_grad=1, device=cpu),
      %onnx::MatMul_12 : Float(138, 96, strides=[1, 138], requires_grad=0, device=cpu),
      %onnx::MatMul_13 : Float(96, 3, strides=[1, 96], requires_grad=0, device=cpu)):
  %onnx::Add_6 : Float(96, strides=[1], device=cpu) = onnx::MatMul[onnx_name="MatMul_0"](%landmarks, %onnx::MatMul_12) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Add[onnx_name="Add_1"](%fc1.bias, %onnx::Add_6) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %onnx::MatMul_8 : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="Relu_2"](%input) # /opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1453:0
  %onnx::Add_10 : Float

In [8]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)
        
        print(klass, labels, klass==labels)
        print(prob)
        print("-----")
        
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([2, 2, 2, 1]) tensor([2, 2, 2, 1]) tensor([True, True, True, True])
tensor([[    0.0000,     0.0420,     0.9580],
        [    0.0000,     0.0124,     0.9876],
        [    0.0061,     0.2028,     0.7911],
        [    0.0172,     0.9746,     0.0082]])
-----
tensor([1, 2, 2, 0]) tensor([1, 2, 2, 0]) tensor([True, True, True, True])
tensor([[    0.1131,     0.8864,     0.0004],
        [    0.0036,     0.0000,     0.9964],
        [    0.0072,     0.2041,     0.7887],
        [    0.9309,     0.0691,     0.0000]])
-----
tensor([1, 1, 0, 2]) tensor([1, 1, 0, 2]) tensor([True, True, True, True])
tensor([[    0.0160,     0.9722,     0.0118],
        [    0.0738,     0.9080,     0.0181],
        [    0.7804,     0.2196,     0.0000],
        [    0.0058,     0.1920,     0.8022]])
-----
tensor([2, 2, 1, 1]) tensor([2, 2, 1, 1]) tensor([True, True, True, True])
tensor([[    0.0000,     0.0070,     0.9930],
        [    0.0084,     0.2320,     0.7596],
        [    0.0476,     0.9480,   

In [9]:
print("Done")

Done
